In [71]:
import numpy as np
import pandas as pd
import requests
import json
import keyring
import time
import datetime
from requests.structures import CaseInsensitiveDict
import matplotlib.pyplot as plt
import plotly.express as px

* [Docs for the API](https://certificationapi.oshwa.org/documentation#section/Introduction)
* [The website version is here](https://certification.oshwa.org/list.html)
* Step one, get a token!
    * Fill out the form [here](https://certificationapi.oshwa.org/)
    * There is a pop-up that will have a big long string of letters and nubmers

* Step two, let's add that token to our our keyring
    * On Linux install keyring with, `sudo apt install keyring`
    * Copy the api key and then add it to keyring by running  `keyring set oshwa_api NoName` and pasting the api key
    * To check the keyring run `keyring set oshwa_api NoName`
    * Add it to your password manager if you use one. 
* Next we'll add the keyring safely to our notebook

In [2]:
# We'll get our first key
oshwa_key = keyring.get_password("oshwa_api","NoName")
if oshwa_key is None:
    print('Failed to get API key')
else:
    print("Got API Key")

Got API Key


# Now we're going to make our first request. We'll need our endpoint and a token.
# [This tool is really helpful for figuring out request formats](https://reqbin.com/req/python/5k564bhv/get-request-with-bearer-token-authorization-header) 

In [3]:
url = "https://certificationapi.oshwa.org/api/projects/"
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
headers["Authorization"] = "Bearer {0}".format(oshwa_key)
params = {"limit":1500,"page":3}
resp = requests.get(url, headers=headers, params=params)
print(resp.status_code)
if resp.status_code == 200:
    cert_list = resp.json()
    print(len(cert_list))

200
1000


In [4]:
# Now we're going to convert this data to a pandas dataframe and clean up the time
df = pd.DataFrame(data=cert_list)
df.to_csv("RawData.csv")
df['certificationDate'] = pd.to_datetime(df['certificationDate'], format="%Y-%m-%d %H:%M:%S")
df.head()

,oshwaUid,responsibleParty,country,publicContact,projectName,projectWebsite,projectVersion,previousVersions,projectDescription,primaryType,additionalType,projectKeywords,citations,documentationUrl,hardwareLicense,softwareLicense,documentationLicense,certificationDate
0,ID000008,SENTSOR ELECTRONICS,Indonesia,sentsor.tech@gmail.com,SENTSOR CORE BOARD ESP32-DEV,https://github.com/adamalfath/sentsor-core-esp...,2.0-DEV,"[{'metadata': {'tags': []}, 'fields': {'oshwaU...",General purpose ESP32 development board with o...,Electronics,"[Electronics, IOT]","[SENTSOR, DEVELOPMENT BOARD, IOT, ESP32, D...",[],https://github.com/adamalfath/sentsor-core-esp...,Other,No software,CC BY-SA,2021-03-26 00:00:00-04:00
1,ID000007,SENTSOR ELECTRONICS,Indonesia,sentsor.tech@gmail.com,SENTSOR CORE BOARD ESP32-EMBED,https://github.com/adamalfath/sentsor-core-esp...,2.0-EMBED,"[{'metadata': {'tags': []}, 'fields': {'oshwaU...",Super low power ESP32 development board for em...,Electronics,"[Electronics, IOT]","[SENTSOR, DEVELOPMENT BOARD, IOT, ESP32, D...",[],https://github.com/adamalfath/sentsor-core-esp...,Other,No software,CC BY-SA,2021-03-26 00:00:00-04:00
2,DE000093,bmc::labs GmbH,Germany,info@bmc-labs.com,mini::base,https://bmc-board.com/,1.0.0,NaN,the bmc::board project consits of a microcontr...,Electronics,NaN,"[STM32, bmc::mini, bmc::board]",[],https://github.com/bmc-labs/board_mini,CERN,MIT,CC BY-SA,2020-11-13 00:00:00-05:00
3,US001096,Mike Diehl,United States of America,diehl.mike.a@gmail.com,HamWing,https://github.com/W8LID/HamWing,1.0,NaN,The HamWIng FeatherWing provides an easy to us...,Electronics,NaN,"[amateur radio, adafruit, feather, featherw...",[],https://github.com/W8LID/HamWing,Solderpad,LGPL,CC BY-SA,2021-03-25 00:00:00-04:00
4,DE000104,Conor Burns,Germany,mail@0xcb.dev,0xCB,https://github.com/conor-burns/0xcb-1337,1.0,NaN,Open Source PCB for a QMK macropad,Electronics,"[3D Printing, Electronics, Manufacturing]","[macropad, qmk, arduino, keyboard]",[{'url': 'https://github.com/qmk/qmk_firmware'...,https://github.com/Conor-Burns/0xcb-1337/blob/...,CERN,GPL,CC BY,2021-03-24 00:00:00-04:00


In [5]:
# Let's now just pick certifications from 2020
start = datetime.datetime(2020,1,1,tzinfo=datetime.timezone.utc)
stop = datetime.datetime(2020,12,31,tzinfo=datetime.timezone.utc)
df2020 = df.loc[(df['certificationDate'] >= start) & 
                (df['certificationDate'] <  stop)] 
print("{0} projects from 2020.".format(len(df2020)))

762 projects from 2020.


In [6]:
countries = df2020.country.unique()
print("Unique countries {0} -- {1}".format(len(countries),countries))
parties = df2020.responsibleParty.unique()
print("Unique Entities {0} -- {1}".format(len(parties),df2020.responsibleParty.unique()))

Unique countries 32 -- ['Germany' 'United States of America' 'Croatia' 'Mauritius' 'Switzerland'
 'Indonesia' 'United Kingdom' 'Brazil' 'Sweden' 'India' 'Guatemala'
 'Australia' 'Spain' 'China' 'Ireland' 'Czech Republic' 'Bulgaria'
 'Russia' 'Canada' 'Thailand' 'Mexico' 'Netherlands' 'France' 'Lithuania'
 'Costa Rica' 'El Salvador' 'Finland' 'Taiwan' 'Greece' 'Portugal'
 'Sri Lanka' 'Poland']
Unique Entities 106 -- ['bmc::labs GmbH' 'System76' 'TAVU d.o.o.' 'SparkFun Electronics'
 'TECKNOLOGG' 'Whitebox Labs' 'SENTSOR ELECTRONICS' 'Nebra Ltd'
 'Franzininho' 'Neil Hardy' 'Centro de Tecnologia Acadêmia (IF/UFRGS)'
 'Keith Burzinski' 'Thore Krug' 'Anuradha Reddy'
 'Adafruit Industries, LLC' 'Upside Down Labs' 'Angel Isidro'
 'Freetronics' 'Blackhand Dynamics, SL' '深圳市嘉禾三维科技有限公司' 'Brian Lough'
 'Watterott electronic' 'Prusa Research a.s.' 'HiMinds' 'Dimitar Dimitrov'
 'Luxonis Holding Corporation' 'Andrey Lamchenko' 'BIEM ETC design'
 'Thorsten Jaeger' 'OLIMEX Ltd' 'Jacob Proctor' 'Maker a

In [60]:
# Get the number of certs by country
rp_data = df2020.groupby("responsibleParty").count().sort_values("oshwaUid",ascending=False)

In [62]:
rp_data = rp_data[rp_data["oshwaUid"]>1]
fig = px.bar(rp_data,y="oshwaUid",title="2020 Certifications by Origin")
fig.show()

In [70]:
rp_data = df2020.groupby("responsibleParty").count()
fig = px.histogram(rp_data,x="oshwaUid",nbins=10,title="2020 Certifications by Origin")
fig.show()

In [38]:
px.bar?